# Spam Detection in YouTube Comments

#nlp #classification

Your task is to classify YouTube comments into spam and ham categories. You need to use something other than bag of words and Naive Bayes as your models (but it can be a baseline).

Dataset source: https://archive.ics.uci.edu/ml/datasets/YouTube+Spam+Collection.

Assignment: https://is.muni.cz/auth/el/fi/jaro2021/IB031/um/cviceni/assignment.pdf?predmet=1323750

### Imports

In [1]:
# Local
from ipynb.fs.full.data_loader import load_train_test_data, load_train_test_all_cols_data
from ipynb.fs.full.transformers import (FeatureSelector, WordEmbeddingsSeries, 
                                        WordEmbeddingsDF, ExplorativeTransformer, Debugger)
from ipynb.fs.full.utils import print_report

# Math and data stuff
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# Sklearn
from sklearn.pipeline import make_pipeline
from sklearn.compose import make_column_transformer, make_column_selector
from sklearn.dummy import DummyClassifier
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer, HashingVectorizer
from sklearn.naive_bayes import GaussianNB, ComplementNB, BernoulliNB, MultinomialNB, CategoricalNB
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.svm import SVC, SVR
from sklearn.tree import DecisionTreeClassifier

# Tensorflow Keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import GaussianNoise, LSTM, Bidirectional, Dropout, Dense
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier

# Language stuff
from pymagnitude import Magnitude, MagnitudeUtils

# Other
import html

Before:  <a href="http://www.youtube.com/watch?v=KQ6zr6kCPj8&amp;t=2m19s">2:19</a> best part﻿
After:  <a href="http://www.youtube.com/watch?v=KQ6zr6kCPj8&t=2m19s">2:19</a> best part﻿
Before: <a href="http://www.youtube.com/watch?v=KQ6zr6kCPj8&amp;t=2m19s">2:19</a> best part﻿
After: anchortag best part﻿
Before: <a href="http://www.youtube.com/watch?v=KQ6zr6kCPj8&amp;t=2m19s">2:19</a> best part﻿
Before: People, here is a new network like FB...
After: people, here is a new network like fb...
Before: subscribe to my channel  /watch?v=NxK32i0HkDs
Before: please like :D https://premium.easypromosapp.com/voteme/19924/616375350﻿
After: subscribe to my channel  urllink
After: please like :D urllink
                                       COMMENT_ID              AUTHOR  \
1447          z13wxtdpeznid12et23ogtd4zoyvzbnoz04        Sonny Carter   
1846  _2viQ_Qnc6-adCzTDLAhqNVQ5hFYcjPyPI5m7pHY4BY         Lizzy Molly   
1304        z13uutbriumnuj3rq04ccbvqlwjuj1srhyk0k        Warcorpse666   
402      

Load data:

In [2]:
data_dir = "data"

psy = pd.read_csv(f"{data_dir}/Youtube01-Psy.csv", parse_dates=["DATE"])
katy = pd.read_csv(f"{data_dir}/Youtube02-KatyPerry.csv", parse_dates=["DATE"])
lmfao = pd.read_csv(f"{data_dir}/Youtube03-LMFAO.csv", parse_dates=["DATE"])
eminem = pd.read_csv(f"{data_dir}/Youtube04-Eminem.csv", parse_dates=["DATE"])
shakira = pd.read_csv(f"{data_dir}/Youtube05-Shakira.csv", parse_dates=["DATE"])

## Exploration analysis

Presentation: https://docs.google.com/presentation/d/1GIoRrhQ2_eERuowLotWscQhx7X4s4A4mwBm3-aGpO_E/edit

The dataset comes from 2017.
It has five datasets composed by 1 956 real messages extracted from five videos that were among the 10 most viewed on the collection period.

| Artist     | Song (our guess)     | Song published on YouTube (date) | Comments range |
|------------|----------------------|----------------------------------|----------------|
| Psy        | Gangnam Style        | 15. 7. 2012                      | 2013–2015      |
| Katy Perry | Roar                 | 5. 9. 2013                       | 2014–2015      |
| LMFAO      | Party Rock Anthem    | 9. 3. 2011                       | 2014–2015      |
| Eminem     | Love The Way You Lie | 5. 8. 2010                       | 2015-2015      |
| Shakira    | Waka Waka            | 5. 6. 2010                       | 2013–2015      |


In [3]:
joined = pd.concat([psy, katy, lmfao, eminem, shakira]).reset_index(drop=True)
joined

,COMMENT_ID,AUTHOR,DATE,CONTENT,CLASS
0,LZQPQhLyRh80UYxNuaDWhIGQYNQ96IuCg-AYWqNPjpU,Julius NM,2013-11-07 06:20:48.000,"Huh, anyway check out this you[tube] channel: ...",1
1,LZQPQhLyRh_C2cTtd9MvFRJedxydaVW-2sNg5Diuo4A,adam riyati,2013-11-07 12:37:15.000,Hey guys check out my new channel and our firs...,1
2,LZQPQhLyRh9MSZYnf8djyk0gEF9BHDPYrrK-qCczIY8,Evgeny Murashkin,2013-11-08 17:34:21.000,just for test I have to say murdev.com,1
3,z13jhp0bxqncu512g22wvzkasxmvvzjaz04,ElNino Melendez,2013-11-09 08:28:43.000,me shaking my sexy ass on my channel enjoy ^_^ ﻿,1
4,z13fwbwp1oujthgqj04chlngpvzmtt3r3dw,GsMega,2013-11-10 16:05:38.000,watch?v=vtaRGgvGtWQ Check this out .﻿,1
...,...,...,...,...,...
1951,_2viQ_Qnc6-bMSjqyL1NKj57ROicCSJV5SwTrw-RFFA,Katie Mettam,2013-07-13 13:27:39.441,I love this song because we sing it at Camp al...,0
1952,_2viQ_Qnc6-pY-1yR6K2FhmC5i48-WuNx5CumlHLDAI,Sabina Pearson-Smith,2013-07-13 13:14:30.021,I love this song for two reasons: 1.it is abou...,0
1953,_2viQ_Qnc6_k_n_Bse9zVhJP8tJReZpo8uM2uZfnzDs,jeffrey jules,2013-07-13 12:09:31.188,wow,0
1954,_2viQ_Qnc6_yBt8UGMWyg3vh0PulTqcqyQtdE7d4Fl0,Aishlin Maciel,2013-07-13 11:17:52.308,Shakira u are so wiredo,0


In [4]:
print(f"There are {len(joined)} total rows in the dataset.")

There are 1956 total rows in the dataset.


The column `CLASS` is the label column we want to predict.

In [5]:
f"There are {len(joined[joined.CLASS == 1])} spams and {len(joined[joined.CLASS == 0])} hams."

'There are 1005 spams and 951 hams.'

We will split the data set at the very beginning not to make the test data dirty.
`final_test_X` and `final_test_y` will be used at the very end to evaluate and compare our models.

In [6]:
joined_X, joined_y = joined.drop(columns="CLASS"), joined.CLASS

random_state = 42

train_X_orig, final_test_X, train_y, final_test_y = train_test_split(
    joined_X, joined_y, test_size=0.2, random_state=random_state
)

train_df, test_df = train_test_split(
    joined, test_size=0.2, random_state=random_state
)

train_df = train_df.copy()

In [7]:
print(f"After putting test data apart, there are {len(train_X_orig)} train data rows.")

After putting test data apart, there are 1564 train data rows.


In [8]:
f"There are {(train_y == 1).sum()} spams and {(train_y == 0).sum()} hams."

'There are 789 spams and 775 hams.'

...which is well balanced, so we do not need any fancy sampling.

In [9]:
train_X_orig.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1564 entries, 836 to 1126
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   COMMENT_ID  1564 non-null   object        
 1   AUTHOR      1564 non-null   object        
 2   DATE        1369 non-null   datetime64[ns]
 3   CONTENT     1564 non-null   object        
dtypes: datetime64[ns](1), object(3)
memory usage: 61.1+ KB


#### Dataset includes the following columns:
- COMMENT_ID - string
- AUTHOR - string
- DATE - date
- CONTENT (comment itself) - string
- CLASS (1 = spam, 0 = ham) - categorical

The column `COMMENT_ID` provides very little (no) value, let's drop it.

In [10]:
train_X = train_X_orig.drop(columns="COMMENT_ID")
train_X

,AUTHOR,DATE,CONTENT
836,Blaze Blaziken,2015-05-20 03:50:29.098,you cant stop the shuffle﻿
1688,Lucia Scarlet,2015-05-22 11:56:53.104,Amazing song﻿
1505,Paul Buxton,2015-05-21 11:12:22.066,Omg! This guy sounds like an american professo...
1650,Tierra Carbon,2015-05-24 14:21:54.411,It was cool the best song ever ﻿
1573,David Bottenberg,NaT,subscribe to my channel /watch?v=NxK32i0HkDs
...,...,...,...
1130,ItsJoey Dash,2014-07-22 10:04:05.755,EVERYONE PLEASE SUBSCRIBE TO MY CHANNEL OR CAN...
1294,louis canellony,NaT,watch youtube video &quot;EMINEM -YTMA artist ...
860,vieshva .d.exodous,2015-05-18 08:38:34.236,Awesome﻿
1459,viviane trinh,2015-05-21 22:35:35.753,i like the lyrics but not to music video﻿


<font color='red'>The `AUTHOR` column can help in classifying spam – spammers may have specific names (e.g. having a "normal" name instead of a username, copying a well-known username etc.).</font>

There are no correlations between columns.

**Hypothesis**: Something that looks like a spam is not always a spam, it depends on the video.
E.g. if the word appears in the video subtitles, it is less likely to be a spam. However, we will not develop this hypothesis any further.

#### Exploration: Missing values? (Eminem is missing a lot of dates)

In [11]:
for dataset, dataset_name in zip([psy, katy, lmfao, eminem, shakira], ["psy", "katy", "lmfao", "eminem", "shakira"]):
    na = dataset.isna().any()
    print(dataset_name)
    for missing, column_name in zip(na, na.index):
        print(f'\t{column_name:10}: {"" if missing else "NO "}missing values')
    

psy
	COMMENT_ID: NO missing values
	AUTHOR    : NO missing values
	DATE      : NO missing values
	CONTENT   : NO missing values
	CLASS     : NO missing values
katy
	COMMENT_ID: NO missing values
	AUTHOR    : NO missing values
	DATE      : NO missing values
	CONTENT   : NO missing values
	CLASS     : NO missing values
lmfao
	COMMENT_ID: NO missing values
	AUTHOR    : NO missing values
	DATE      : NO missing values
	CONTENT   : NO missing values
	CLASS     : NO missing values
eminem
	COMMENT_ID: NO missing values
	AUTHOR    : NO missing values
	DATE      : missing values
	CONTENT   : NO missing values
	CLASS     : NO missing values
shakira
	COMMENT_ID: NO missing values
	AUTHOR    : NO missing values
	DATE      : NO missing values
	CONTENT   : NO missing values
	CLASS     : NO missing values


We can see that only eminem dataset has missing valued and they are in the `DATE` column.

In [12]:
f"{eminem.DATE.isna().sum()} out of {len(eminem)} DATE values in eminem dataset are missing."

'245 out of 448 DATE values in eminem dataset are missing.'

First, we wanted to replace `NaN` dates with previous `non NaN` date in a sorted dataset, but...

In [13]:
eminem[eminem.DATE.isna()]["CLASS"].value_counts()

1    245
Name: CLASS, dtype: int64

<font color='red'>All Eminem comments that do not have DATE are spams. The information that DATE is missing is therefore valuable.</font>

#### Exploration: What range are comment dates in?

In [14]:
for dataset, dataset_name in zip([psy, katy, lmfao, eminem, shakira], ["psy", "katy", "lmfao", "eminem", "shakira"]):
    sorted_dataset = dataset[~dataset.DATE.isna()].DATE.sort_values()
    earliest = sorted_dataset.iloc[0].year
    latest = sorted_dataset.iloc[-1].year
    print(f"{dataset_name}: {earliest}-{latest}")
    

psy: 2013-2015
katy: 2014-2015
lmfao: 2014-2015
eminem: 2015-2015
shakira: 2013-2015


#### Hypothesis: Longer comment implies spam.

In [15]:
y_pred = train_X.CONTENT.str.len() >= 50
accuracy_score(train_y, y_pred)

0.6304347826086957

...better than random.

#### Hypothesis: Spams contain "check" (e.g. "Check out my channel")

In [16]:
y_pred = train_X.CONTENT.str.contains("check")
accuracy_score(train_y, y_pred)

0.5818414322250639

...better than random.

#### Exploration: Are there any duplicate comments based on ID?

In [17]:
train_df[train_df.COMMENT_ID.duplicated(keep=False)]

,COMMENT_ID,AUTHOR,DATE,CONTENT,CLASS
1421,LneaDw26bFvPh9xBHNw1btQoyP60ay_WWthtvXCx37s,janez novak,NaT,share and like this page to win a hand signed ...,1
1441,LneaDw26bFuH6iFsSrjlJLJIX3qD4R8-emuZ-aGUj0o,Amir bassem,NaT,if u love rihanna subscribe me,1
1420,LneaDw26bFvPh9xBHNw1btQoyP60ay_WWthtvXCx37s,janez novak,NaT,share and like this page to win a hand signed ...,1
1797,_2viQ_Qnc68fX3dYsfYuM-m4ELMJvxOQBmBOFHqGOk0,tyler sleetway,2013-10-05 00:57:25.078,so beutiful,0
1798,_2viQ_Qnc68fX3dYsfYuM-m4ELMJvxOQBmBOFHqGOk0,tyler sleetway,2013-10-05 00:57:25.078,so beutiful,0
1443,LneaDw26bFuH6iFsSrjlJLJIX3qD4R8-emuZ-aGUj0o,Amir bassem,NaT,if u love rihanna subscribe me,1


<font color='red'>We want to drop duplicate rows.</font>

In [18]:
f"We lose only {len(train_df) - len(train_df.drop_duplicates())} rows by dropping duplicates."

'We lose only 3 rows by dropping duplicates.'

Are there any duplicate comments based on Author name?

In [19]:
duplicate_authors = train_df[train_df.duplicated(subset=["AUTHOR"], keep=False)]
duplicate_authors

,COMMENT_ID,AUTHOR,DATE,CONTENT,CLASS
192,z13xtdlovm2hzl05d04ccz1pnvqtezdriqc0k,Uroš Slemenjak,2014-11-07 12:08:13.000,"People, here is a new network like FB...you re...",1
462,z12wj5g52rzbcvprl04cenuj1yyifhxq3hw,LuckyMusiqLive,2014-09-15 17:47:57.000,Katy has the voice of gold. this video really ...,1
141,z12udxjwpwurtlwz304ccbrhdtusth4herk0k,PacKmaN,2014-11-05 21:56:39.000,check men out i put allot of effort into my mu...,1
1828,_2viQ_Qnc6_umVgV0fI-CSScDHuFxNHIVvezCGhajW8,Alain Bruno,2013-10-02 04:01:20.922,Shakira is very beautiful,0
1131,z12dw3tbbzm2gpzty22gtf1bvviqeha2j,ItsJoey Dash,2014-07-22 10:04:00.700,EVERYONE PLEASE SUBSCRIBE TO MY CHANNEL OR CAN...,1
...,...,...,...,...,...
769,z13jvndqyr2zjzpxo04cij4pdva4yhqzlq40k,D Maw,2015-05-25 05:43:54.048,Remeber when this song was good﻿,0
1332,LneaDw26bFuUfz6WhRf9xiRHsxHm0t4fXYLbcPhB7Lk,Scott Johnson,NaT,You guys should check out this EXTRAORDINARY w...,1
1638,z132svd4fvq1wntfd221w5szfzezjri2r,Abdullah Fawzi,2015-05-25 06:23:24.405,"see this<br /><a href=""http://adf.ly"">http://a...",1
1724,z13qfn5yusqoslnn222dutdw4yqmhzhej,LiveLikeLien x,2015-05-20 00:57:56.444,"It makes me happy instantly, and makes me forg...",0


In [20]:
train_df[train_df.AUTHOR == "Uroš Slemenjak"]

,COMMENT_ID,AUTHOR,DATE,CONTENT,CLASS
192,z13xtdlovm2hzl05d04ccz1pnvqtezdriqc0k,Uroš Slemenjak,2014-11-07 12:08:13,"People, here is a new network like FB...you re...",1
658,z13ocdbaxwqdvjnwx04ccz1pnvqtezdriqc0k,Uroš Slemenjak,2014-11-07 12:16:21,"People, here is a new network like FB...you re...",1


In [21]:
duplicate_authors["CLASS"].value_counts()

1    149
0     35
Name: CLASS, dtype: int64

In [22]:
spams, hams = duplicate_authors["CLASS"].value_counts()
f"{(spams / (spams + hams)) * 100:.02f}% of comments from authors who posted multiple comments are spams."

'80.98% of comments from authors who posted multiple comments are spams.'

Does the same author post both spams and hams? If so, are the labels truly correct?

In [23]:
grouped_by_author = duplicate_authors.groupby("AUTHOR").CLASS.mean()
posted_both_spam_and_ham = (grouped_by_author != 0) & (grouped_by_author != 1)
f"There are {len(grouped_by_author[posted_both_spam_and_ham])} authors who posted both spam and ham. But this might not be generally "

'There are 0 authors who posted both spam and ham. But this might not be generally '

In [24]:
duplicate_comments = train_df[train_df.duplicated(subset=["CONTENT"], keep=False)]
grouped_by_content = duplicate_comments.groupby("CONTENT").CLASS.mean()
same_content_spam_and_ham = (grouped_by_content != 0) & (grouped_by_content != 1)
f"There are {len(grouped_by_content[same_content_spam_and_ham])} comments that are classified both as spam and ham."

'There are 0 comments that are classified both as spam and ham.'

But this might be true only for the data we are working with, and not truth in general.

In [25]:
test_df[test_df.AUTHOR == "Connor Mire"]

,COMMENT_ID,AUTHOR,DATE,CONTENT,CLASS
810,z13xxf3qlq2bxpm1o22zidpqbn2tfpcjr04,Connor Mire,2015-05-21 20:46:54.704,This Song is AWESOME!!!!﻿,0
809,z13uzb3zqoylw1kl022zidpqbn2tfpcjr04,Connor Mire,2015-05-21 20:47:08.673,I&#39;m A SUBSCRIBER﻿,1


We can see that *Connor Mire* posted both spam and ham. And the labels are correct.

In [26]:
spams, hams = duplicate_comments["CLASS"].value_counts()
f"{(spams / (spams + hams)) * 100:.02f}% of duplicate comments are spams."

'77.30% of duplicate comments are spams.'

We do not need to transform emojis, some word embeddings handle them well:

<div class="alert alert-block alert-danger">Will download <b>1.6 GB</b> large word embeddings model.</div>

In [27]:
vectors = Magnitude(MagnitudeUtils.download_model('fasttext/medium/wiki-news-300d-1M'))
print(vectors.similarity(":)", ":D"))

0.5047033597406592


## Feature engineering

In [28]:
# One-hot-encode AUTHOR column
pipeline = make_pipeline(
    make_column_transformer(
        (OneHotEncoder(), ["AUTHOR"]),
        remainder="passthrough",
        sparse_threshold=0
    )
)
# DEMO
pipeline.fit(train_X, train_y)
pipeline.transform(train_X[:1])

array([[0.0, 0.0, 0.0, ..., 0.0, Timestamp('2015-05-20 03:50:29.098000'),
        'you cant stop the shuffle\ufeff']], dtype=object)

## Possible models and techniques to use:
- [Bag of words](https://en.wikipedia.org/wiki/Bag-of-words_model)
- [Naive Bayes](https://www.kaggle.com/mohammedakhil/youtube-spam-filter-using-naive-bayes)
- [LSTM](https://keras.io/api/layers/recurrent_layers/lstm/)
- [DecisionTreeClassifier](https://datascience.stackexchange.com/questions/67250/decision-tree-in-sentiment-analysis)
- [Magnitude Word Embeddings](https://colab.research.google.com/drive/1lOcAhIffLW8XC6QsKzt5T_ZqPP4Y9eS4#scrollTo=95Xg9EyU-ZYr)

## Baseline models

In [29]:
train_X, test_X, train_y, test_y = load_train_test_data()

### Dummy Classifier

In [30]:
for strategy, strategy_name in zip(["uniform", "stratified", "most_frequent"],
                                   ["completely random", "proportional", "most frequent"]):
    dummy_clf = DummyClassifier(strategy=strategy)
    dummy_clf.fit(train_X, train_y)
    print(f"{strategy_name}: {dummy_clf.score(test_X, test_y):.2f}") # ignores the test data

completely random: 0.44
proportional: 0.50
most frequent: 0.48


### Bag of words + Naive Bayes

In [31]:
N_FEATURES = len(CountVectorizer().fit(train_X).vocabulary_)*10

vectorizers = [("Count vect.", CountVectorizer()), 
               ("TF-IDF vect.", TfidfVectorizer()), 
               ("Count binary vect.", CountVectorizer(binary=True)),
               ("Hashing vect.", HashingVectorizer(n_features = N_FEATURES, alternate_sign=False)),
              ]
bayeses = [
    ("ComplementNB", ComplementNB()), 
    ("BernoulliNB", BernoulliNB()), 
    ("MultinomialNB", MultinomialNB())
]

for vectorizer_name, vectorizer in vectorizers:
    for bayes_name, bayes in bayeses:
        pipeline= make_pipeline(
            vectorizer,
            bayes
        )

        pipeline.fit(np.array(train_X), train_y)
        y_pred = pipeline.predict(test_X)

        print(f"{vectorizer_name}; {bayes_name}: {pipeline.score(test_X, test_y):.2f}")

        tn, fp, fn, tp = confusion_matrix(test_y, y_pred).ravel()
        print(f"{vectorizer_name}; {bayes_name} - FP: {fp}, FN: {fn}")

Count vect.; ComplementNB: 0.95
Count vect.; ComplementNB - FP: 8, FN: 8
Count vect.; BernoulliNB: 0.89
Count vect.; BernoulliNB - FP: 4, FN: 29
Count vect.; MultinomialNB: 0.94
Count vect.; MultinomialNB - FP: 10, FN: 8
TF-IDF vect.; ComplementNB: 0.94
TF-IDF vect.; ComplementNB - FP: 8, FN: 10
TF-IDF vect.; BernoulliNB: 0.89
TF-IDF vect.; BernoulliNB - FP: 4, FN: 29
TF-IDF vect.; MultinomialNB: 0.94
TF-IDF vect.; MultinomialNB - FP: 10, FN: 10
Count binary vect.; ComplementNB: 0.95
Count binary vect.; ComplementNB - FP: 8, FN: 9
Count binary vect.; BernoulliNB: 0.89
Count binary vect.; BernoulliNB - FP: 4, FN: 29
Count binary vect.; MultinomialNB: 0.94
Count binary vect.; MultinomialNB - FP: 10, FN: 9
Hashing vect.; ComplementNB: 0.91
Hashing vect.; ComplementNB - FP: 21, FN: 7
Hashing vect.; BernoulliNB: 0.91
Hashing vect.; BernoulliNB - FP: 6, FN: 23
Hashing vect.; MultinomialNB: 0.90
Hashing vect.; MultinomialNB - FP: 24, FN: 7


## Our models

### Word embeddings + Keras LSTM

In [32]:
MAX_WORDS = 30 # he maximum number of words the sequence model will consider
STD_DEV = 0.01 # Deviation of noise for Gaussian Noise applied to the embeddings
HIDDEN_UNITS = 100 # The number of hidden units from the LSTM
DROPOUT_RATIO = .8 # The ratio to dropout
BATCH_SIZE = 100 # The number of examples per train/validation step
EPOCHS = 50 # The number of times to repeat through all of the training data
LEARNING_RATE = .01 # The learning rate for the optimizer

<div class="alert alert-block alert-danger">Will download <b>1.6 GB</b> large word embeddings model.</div>

In [33]:
vectors = Magnitude(MagnitudeUtils.download_model('fasttext/medium/wiki-news-300d-1M'), pad_to_length = MAX_WORDS)

In [34]:
train_X, test_X, train_y, test_y = load_train_test_data()

**GaussianNoise** - useful to mitigate overfitting (could be used as a form of random data augmentation). Gaussian Noise (GS) is a natural choice as corruption process for real valued inputs.
As it is a regularization layer, it is only active at training time.

**Bidirectional** – sequence processing model that consists of two LSTMs: one taking the input in a forward direction, and the other in a backwards direction. Bidirectional LSTMs effectively increase the amount of information available to the network, improving the context available to the algorithm

**Dropout** – a stochastic regularization technique and should reduce overfitting by (theoretically) combining many different neural network architectures.
With Dropout, the training process essentially drops out neurons in a neural network.

**Dense** - The dense layer is a neural network layer that is connected deeply, which means each neuron in the dense layer receives input from all neurons of its previous layer.
In the background, the dense layer performs a matrix-vector multiplication. The values used in the matrix are actually parameters that can be trained and updated with the help of backpropagation.

In [35]:
def create_model():
    model = Sequential()

    model.add(GaussianNoise(STD_DEV, input_shape=(MAX_WORDS, vectors.dim)))
    model.add(Bidirectional(LSTM(HIDDEN_UNITS, activation='tanh'), merge_mode='concat'))
    model.add(Dropout(DROPOUT_RATIO))
    model.add(Dense(1, activation='sigmoid'))
    
    model.compile(
        loss='binary_crossentropy',
        optimizer=Adam(lr=LEARNING_RATE),
        metrics=['accuracy'])
    
    return model

In [36]:
clf = make_pipeline(
    WordEmbeddingsSeries(vectors),
    KerasClassifier(create_model, epochs=5, batch_size=32, validation_split=0.1)
)

clf.fit(train_X, train_y)
clf.score(test_X, test_y)

Epoch 1/5
36/36 [==============================] - 5s 73ms/step - loss: 0.6232 - accuracy: 0.6861 - val_loss: 0.8157 - val_accuracy: 0.6880
Epoch 2/5
36/36 [==============================] - 1s 30ms/step - loss: 0.3479 - accuracy: 0.8688 - val_loss: 0.2045 - val_accuracy: 0.9440
Epoch 3/5
36/36 [==============================] - 1s 32ms/step - loss: 0.2534 - accuracy: 0.9092 - val_loss: 0.1371 - val_accuracy: 0.9680
Epoch 4/5
36/36 [==============================] - 1s 32ms/step - loss: 0.2061 - accuracy: 0.9382 - val_loss: 0.1995 - val_accuracy: 0.9040
Epoch 5/5
10/10 [==============================] - 0s 12ms/step - loss: 0.3128 - accuracy: 0.8339


0.8338658213615417

In [37]:
lstm_pred_y = clf.predict(test_X)
print_report(test_y, lstm_pred_y)

/Users/petr.janik/opt/anaconda3/envs/ib031/lib/python3.8/site-packages/tensorflow/python/keras/engine/sequential.py:450: UserWarning: `model.predict_classes()` is deprecated and will be removed after 2021-01-01. Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
  warnings.warn('`model.predict_classes()` is deprecated and '


Confusion Matrix
[[160   3]
 [ 49 101]]

Classification Report
              precision    recall  f1-score   support

         Ham       0.77      0.98      0.86       163
        Spam       0.97      0.67      0.80       150

    accuracy                           0.83       313
   macro avg       0.87      0.83      0.83       313
weighted avg       0.86      0.83      0.83       313



## Support vector Classifier

In [38]:
train_X, test_X, train_y, test_y = load_train_test_all_cols_data()

In [39]:
svc = make_pipeline(
    ExplorativeTransformer(),
    FeatureSelector(["SUSPICIOUS_WORDS_COUNT", "NULL_IN_DATE_TIME", "HAS_LINK", "NOT_UNIQUE_AUTHOR"]),
    SVC(),
)

svc.fit(train_X, train_y)
svc_pred_y = svc.predict(test_X)
print("SVC accuracy score :" + str(accuracy_score(test_y, svc_pred_y)))

SVC accuracy score :0.8785942492012779


In [40]:
print_report(test_y, svc_pred_y)

Confusion Matrix
[[159   4]
 [ 34 116]]

Classification Report
              precision    recall  f1-score   support

         Ham       0.82      0.98      0.89       163
        Spam       0.97      0.77      0.86       150

    accuracy                           0.88       313
   macro avg       0.90      0.87      0.88       313
weighted avg       0.89      0.88      0.88       313



## Decision Tree Classifier

Steps:
* splitting the space by setting rules
* removing the unnecessary splits
* using the class with majority votes as the prediction

Decision tree first splits the data based on these concepts: Pure and Impure, Impurity measurement, Information Gain.
This is calculated when traversing trough the tree, then edge with more gain is used to decide about the categorization of an instance
given to the algorithm.

In [41]:
from sklearn.compose import ColumnTransformer
dtc = make_pipeline(
    AddContainsCheckColumn(),
    WordEmbeddingsDF(["AUTHOR", "CONTENT"]),
    FeatureSelector(columns=["CONTAINS_CHECK"], dtype_include="float32"),
    Debugger(),
    DecisionTreeClassifier()
)

dtc.fit(train_X, train_y)
print(f"DTC accuracy score: {dtc.score(test_X, test_y)}")

NameError: name 'AddContainsCheckColumn' is not defined

In [ ]:
dtc_pred_y = dtc.predict(test_X)
print_report(test_y, dtc_pred_y)

## Summary:
**Baseline MNBayes** - 0.9–0.95

**LSTM** - 0.93

This model uses Magnitude library to convert words into vectors of numbers (word embeddings). This conversion, combined with LSTM, preserves the order of words.
However, MLP with CountVectorizer, which does not preserve the order performed slightly better (0.95). This iundicates that the architecture and hyperparameters of the used LSTM still need some fine tuning.

**SVC** - 0.87

This result was reached with our state-of-the-art ExplorationTransformer. The SVM classifier was also tried with a Count Vectorizer and, interestingly, it performed better with stop words included than without them (0.94 vs 0.92).

**Tree Classifier** - 0.95

Considering the relative simplicity of this model, it performed rather well on the data set.

We have prepared many transformers, which could be used to create new columns. 
We could then try different combinations of these columns and select the best, perhaps by using SelectKBest.

However, considering the success of even the baseline model, and the success of other models on just the vectorized comments, the performance on this
dataset might increase only marginally.